In [22]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import glob
import os

In [23]:
all_files = glob.glob("comodity-price-prediction-penyisihan-arkavidia-9/Google Trend/" + "*.csv")

In [84]:
lst = []



for file in all_files:
    df = pd.read_csv(file)

    commodity_name = os.path.basename(os.path.splitext(file)[0])
    # print(commodity_name)

    # print(df[["Date", "Price", "Change %"]])

    df = df.rename(columns={"Price": f"Price_{commodity_name}", "Change %": f"Change_{commodity_name}"})



    
    lst.append(df[["Date", f"Price_{commodity_name}", f"Change_{commodity_name}"]])


merged_df = pd.DataFrame({"Date": []})

for df in lst:

    merged_df = pd.merge(right=merged_df, left=df, how="outer", on="Date")

    print(merged_df)


           Date  Price_US Sugar 11 Futures Historical Data  \
0    2022-01-03                                      18.74   
1    2022-01-04                                      18.75   
2    2022-01-05                                      18.34   
3    2022-01-06                                      18.19   
4    2022-01-07                                      18.05   
..          ...                                        ...   
684  2024-09-24                                      23.12   
685  2024-09-25                                      23.42   
686  2024-09-26                                      23.31   
687  2024-09-27                                      22.79   
688  2024-09-30                                      22.67   

    Change_US Sugar 11 Futures Historical Data  
0                                       -0.74%  
1                                        0.05%  
2                                       -2.19%  
3                                       -0.82%  
4         

In [85]:
merged_df

,Date,Price_Newcastle Coal Futures Historical Data,Change_Newcastle Coal Futures Historical Data,Price_Palm Oil Futures Historical Data,Change_Palm Oil Futures Historical Data,Price_Crude Oil WTI Futures Historical Data,Change_Crude Oil WTI Futures Historical Data,Price_US Wheat Futures Historical Data,Change_US Wheat Futures Historical Data,Price_Natural Gas Futures Historical Data,Change_Natural Gas Futures Historical Data,Price_US Sugar 11 Futures Historical Data,Change_US Sugar 11 Futures Historical Data
0,2022-01-03,157.50,-7.13%,"5,259.00",1.94%,75.85,1.30%,758.00,-1.65%,3.678,3.40%,18.74,-0.74%
1,2022-01-04,174.10,10.54%,"5,308.00",0.93%,76.74,1.17%,770.00,1.58%,3.582,-2.61%,18.75,0.05%
2,2022-01-05,179.90,3.33%,"5,437.00",2.43%,77.47,0.95%,760.75,-1.20%,3.710,3.57%,18.34,-2.19%
3,2022-01-06,196.50,9.23%,"5,387.00",-0.92%,78.88,1.82%,746.00,-1.94%,3.670,-1.08%,18.19,-0.82%
4,2022-01-07,195.90,-0.31%,"5,330.00",-1.06%,78.44,-0.56%,758.50,1.68%,3.726,1.53%,18.05,-0.77%
...,...,...,...,...,...,...,...,...,...,...,...,...,...
726,2024-09-24,139.50,0.00%,"4,074.00",0.62%,70.84,1.77%,578.00,-0.77%,2.791,-2.21%,23.12,2.53%
727,2024-09-25,139.75,0.18%,"4,125.00",1.25%,69.13,-2.41%,589.25,1.95%,2.817,0.93%,23.42,1.30%
728,2024-09-26,139.60,-0.11%,"4,235.00",2.67%,67.19,-2.81%,584.25,-0.85%,2.753,-2.27%,23.31,-0.47%
729,2024-09-27,139.75,0.11%,"4,159.00",-1.79%,67.67,0.71%,580.00,-0.73%,3.304,20.01%,22.79,-2.23%


In [86]:
merged_df.to_csv("comodity-price-prediction-penyisihan-arkavidia-9/global_commodity_price_merged.csv", index=False)